In [1]:
# !pip install pymysql

  Using cached PyMySQL-1.0.2-py3-none-any.whl (43 kB)


In [34]:
import pymysql

In [35]:
con = pymysql.connect(
    host = 'localhost', 
    port = 3306, # int값
    user = 'root', 
    password = 'woody1234', 
    db = 'mldb3', 
    charset = 'utf8'
)

con # stream: 통로
print('1. 연결성공')

1. 연결성공


In [36]:
cur = con.cursor()
cur # 통로 안 객체 가리킴
print('2. 커서 객체 획득')

2. 커서 객체 획득


In [37]:
# sql = 'select * from member'
sql = """
    SELECT m.name, m.tel, b.content
    FROM board b
    INNER JOIN member m ON m.id = b.writer
    WHERE content LIKE "%java"
    """

In [38]:
result_count = cur.execute(sql) # 객체 row 갯수 가져옴

print('3. sql 전송 성공', result_count)

3. sql 전송 성공 2


In [39]:
rows = cur.fetchall()
print('4. 검색결과 ------------')
print(rows)

4. 검색결과 ------------
(('apple', '555555555', 'fun java'), ('apple', '555555555', 'good java'))


In [40]:
# 메모리가 잡혀있는 상태이므로 연결 끊어줘야 함
con.close()

In [41]:
for index, row in enumerate(rows):
    print(index, row)
#     print(index, row[0])

0 ('apple', '555555555', 'fun java')
1 ('apple', '555555555', 'good java')
